In [25]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import numpy as np
import os
from reload_recursive import reload_recursive
import helpers
import pyperclip
import statsmodels.api as sm
import subprocess

from mri_data import file_manager as fm
from mri_data import utils

In [26]:
drive_root = fm.get_drive_root()
msmri_home = Path("/home/srs-9/Projects/ms_mri")
msmri_datadir = msmri_home / "data"
curr_dir = Path("/home/srs-9/Projects/ms_mri/analysis/paper1")
data_dir = curr_dir / "data0"
showfigs = False
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.display.precision = 3

In [27]:
df1 = pd.read_csv(data_dir / "t1_data_full.csv")
df1.set_index('subid', inplace=True)
df1 = helpers.set_dz_type5(df1)

df2 = pd.read_csv(data_dir / "t1_2_data_full.csv")
df2.set_index('subid', inplace=True)
df2 = helpers.set_dz_type5(df2)

df3 = pd.read_csv(data_dir / "t1_3_data_full.csv")
df3.set_index('subid', inplace=True)
df3 = helpers.set_dz_type5(df3)

df_flair = pd.read_csv(data_dir / "flair_data_full.csv")
df_flair.set_index('subid', inplace=True)
df_flair = helpers.set_dz_type5(df_flair)

df_t1_flair = pd.read_csv(data_dir / "t1_flair_data_full.csv")
df_t1_flair.set_index('subid', inplace=True)
df_t1_flair = helpers.set_dz_type5(df_t1_flair)

In [28]:
df = pd.DataFrame(index=df1.index)
df['age'] = df1['age']
df['dz_type5'] = df1['dz_type5']
df['pineal_volume1'] = df1['pineal_volume']
df['pineal_volume2'] = df2['pineal_volume']
df['EDSS'] = df1['extracted_EDSS']
df['choroid_volume1'] = df1['choroid_volume']
df['choroid_volume2'] = df2['choroid_volume']
df['cp_diff'] = df['choroid_volume1'] - df['choroid_volume2']
df['choroid_volume_fl'] = df_flair['choroid_volume']
df['choroid_volume_t1fl'] = df_t1_flair['choroid_volume']


In [29]:
df = df.sort_values(by="cp_diff", ascending=False)
df_pms = df[df['dz_type5']=="PMS"]
df_rms = df[df['dz_type5']=="RMS"]
df_nind = df[df['dz_type5']=="NIND"].sort_values(by="choroid_volume1", ascending=False)
df_oind = df[df['dz_type5']=="OIND"].sort_values(by="choroid_volume1", ascending=False)
df_pms_w = df[(df['dz_type5']=="PMS") & (df1['flair_contrast'] == "WITH")]

/tmp/ipykernel_7333/1759061542.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_pms_w = df[(df['dz_type5']=="PMS") & (df1['flair_contrast'] == "WITH")]


In [30]:
df = df.sort_values(by="pineal_volume2")

In [49]:
sub = 2100

dataroot = drive_root / "3Tpioneer_bids"
labelroot = drive_root / "srs-9/3Tpioneer_bids_predictions"
label1 = labelroot / df1.loc[sub, 'sub-ses'] / df1.loc[sub, 'label']
label2 = labelroot / df2.loc[sub, 'sub-ses'] / df2.loc[sub, 'label']
label_flair = labelroot / df_flair.loc[sub, 'sub-ses'] / df_flair.loc[sub, 'label']
label_t1_flair = labelroot / df_t1_flair.loc[sub, 'sub-ses'] / df_t1_flair.loc[sub, 'label']

t1 = dataroot / df1.loc[sub, 'sub-ses'] / "t1.nii.gz"
t1_gd = dataroot / df1.loc[sub, 'sub-ses'] / "t1_gd.nii.gz"
flair = dataroot / df1.loc[sub, 'sub-ses'] / "flair.nii.gz"

images = []
for im in [t1, t1_gd, flair]:
    if im.is_file():
        images.append(im)

labels = []
for lab in [label_t1_flair, label_flair, label1, label2]:
    if lab.is_file():
        labels.append(lab)
    else:
        man_label = dataroot / df1.loc[sub, "sub-ses"] / df1.loc[sub, "label"]
        if man_label.is_file():
            labels.append(man_label)
        break

cmd = utils.open_itksnap_workspace_cmd(images=images, 
                                       labels=labels, 
                                       win=True)
pyperclip.copy(cmd)
print(cmd)
# subprocess.Popen(cmd.split(" "))

itksnap -g H:/3Tpioneer_bids/sub-ms2100/ses-20180529/t1.nii.gz -o H:/3Tpioneer_bids/sub-ms2100/ses-20180529/t1_gd.nii.gz H:/3Tpioneer_bids/sub-ms2100/ses-20180529/flair.nii.gz -s H:/srs-9/3Tpioneer_bids_predictions/sub-ms2100/ses-20180529/flair.t1_choroid_pineal_pituitary3_pred.nii.gz H:/srs-9/3Tpioneer_bids_predictions/sub-ms2100/ses-20180529/flair_choroid_pineal_pituitary_FLAIR-1_pred.nii.gz H:/srs-9/3Tpioneer_bids_predictions/sub-ms2100/ses-20180529/t1_choroid_pineal_pituitary_T1-1_pred.nii.gz H:/srs-9/3Tpioneer_bids_predictions/sub-ms2100/ses-20180529/t1_choroid_pineal2_pituitary_T1-1_pred.nii.gz


In [8]:
def change_std(col, fact):
    return col.mean() + (col - col.mean())*fact

In [9]:
check = change_std(df2.loc[df2['dz_type5'] == "PMS", "choroid_volume"], 0.9)
print(check.mean())
print(check.std())

2258.625196764706
552.9406768413594


In [10]:
print(df1.loc[df1['dz_type5'] == "NIND", "choroid_volume"].mean())
print(df1.loc[df1['dz_type5'] == "NIND", "choroid_volume"].std())
print(df2.loc[df2['dz_type5'] == "NIND", "choroid_volume"].mean())
print(df2.loc[df2['dz_type5'] == "NIND", "choroid_volume"].std())

1980.1583556976743
671.2784403942993
1931.7214596511628
644.1742227800931


In [ ]:
model_data = df2.copy()
# model_data.loc[model_data['dz_type5']=="PMS", "choroid_volume"] = df.loc[df['dz_type5']=="PMS", "choroid_volume1"]
# subs_to_change = [1453, 1276, 1370, 1049, 1077, 1299, 1155, 2027, 1322]
# subs_to_change = [1530, 1435, 1442, 1106, 1068, 2051, 1052, 1217, 1049, 1508]
# model_data.loc[subs_to_change, "choroid_volume"] = df.loc[subs_to_change, "choroid_volume1"]
fact = 0.7
model_data.loc[model_data['dz_type5']=="PMS", "choroid_volume"] = change_std(model_data.loc[model_data['dz_type5']=="PMS", "choroid_volume"], fact)
model_data.loc[model_data['dz_type5']=="RMS", "choroid_volume"] = change_std(model_data.loc[model_data['dz_type5']=="RMS", "choroid_volume"], fact)

model_data = helpers.fix_edss(model_data)
model_data = helpers.clean_df(model_data)

model_data = model_data[model_data['dz_type5'].isin(["RMS", "PMS"])]
model_data = pd.concat((model_data, pd.get_dummies(model_data["sex"], dtype="int")), axis=1)

model_data['EDSS'] = model_data['EDSS'].astype("float")

res = sm.OLS.from_formula("EDSS ~ choroid_volume + tiv + age + Female", data=model_data).fit()
print(res.summary())

In [ ]:
model_data = df2.copy()
model_data = helpers.fix_edss(model_data)
model_data = helpers.clean_df(model_data)

model_data = model_data[model_data['dz_type5'].isin(["RMS", "PMS", "NIND", "OIND"])]
model_data = pd.concat((model_data, pd.get_dummies(model_data["sex"], dtype="int")), axis=1)
model_data['EDSS'] = model_data['EDSS'].astype("float")
res = sm.OLS.from_formula("EDSS ~ choroid_volume + tiv + age + Female", data=model_data).fit()
print(res.summary())

In [ ]:
influence = res.get_influence()
cooks_d = influence.cooks_distance
cooks_d[1]

In [ ]:
influence.exog.shape

(397, 5)

In [148]:
sub = 1235
t1 = dataroot / df1.loc[sub, 'sub-ses'] / "proc/t1_std.nii.gz"
flair = dataroot / df1.loc[sub, 'sub-ses'] / "proc/flair-brain-mni_reg.nii.gz"
cmd = utils.open_itksnap_workspace_cmd(images=[t1, flair])
subprocess.Popen(cmd.split(" "))

<Popen: returncode: None args: ['itksnap', '-g', '/mnt/h/3Tpioneer_bids/sub-...>

In [156]:
df1.loc[1491, 'flair_contrast']

'WITH'

In [18]:
print(t1)

/media/smbshare/3Tpioneer_bids/sub-ms1370/ses-20210421/proc/t1_std.nii.gz


In [ ]:
fig, axs = plt.subplots(2,1,figsize=(12, 4))

axs[0].hist(df1.loc[df1['dz_type5'].isin(["RMS", "PMS"]), 'choroid_volume'])
axs[1].hist(df3.loc[df2['dz_type5'].isin(["RMS", "PMS"]), 'choroid_volume'])

In [ ]:
plt.scatter(df1.loc[df1['dz_type5'].isin(["RMS", "PMS"]), 'choroid_volume'], 
            df3.loc[df3['dz_type5'].isin(["RMS", "PMS"]), 'choroid_volume'])

In [ ]:
plt.hist((df1.loc[df1['dz_type5'].isin(["PMS"]), 'choroid_volume'] - df2.loc[df2['dz_type5'].isin(["PMS"]), 'choroid_volume']))